In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import division
from collections import Counter
from operator import itemgetter
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

%matplotlib inline

# Preprocess data

In [2]:
#Load the data
data_socio = pd.read_csv('samlede_socio_data_kbh.csv', encoding = 'utf8')

In [3]:
#Delete all the features that are ID's or geometry
dellist=['rode_nr','v_id','FID','rodenavn','wkb_geometry']
#Get a list of all the years. Remove 2014 since the data from that year is bad. Replace it with 'All years'
yearlist=list(set(data_socio['aar']))
yearlist[-1]='All years'

#Work only with percent features. Remove all the ones that just state a number. They come in pairs
#One with percent and one with number
for name in list(data_socio):
    if 'antal' in name: 
        dellist.append(name)

#Create a new dataframe with only the needed variables
MLdf = data_socio.drop(dellist, 1)

# Machine learning - Regression

## Random forest

In [4]:
#Do nfold cross validation
nfold=10
year_error=[]

#Do the random forest (RF) for all the years
for year in yearlist:
    tree_performance_error_RMSE=[]
    treefit=[]
    #Determine what yeark is worked with.
    if year=='All years':
        RFdf=MLdf
    else:
        RFdf=MLdf[MLdf['aar']==year]
    #Use the one below if predicting amount of danes.
    drop_list=['aar','pct_dansk','pct_vestlig','pct_ikke_vestlig']
   
    #Use the one below if predicting high income
    #drop_list=['aar','pct_lav_indkomst','pct_middel_indkomst','pct_hoj_indkomst']
    
    #Use the one below if predicting age above 65
    #drop_list=['aar','alder_pct_0_5','alder_pct_6_17','alder_pct_18_29','alder_pct_30_39','alder_pct_40_49','alder_pct_50_64',
    #           'alder_pct_over_65']
    
    #Drop the feature that is being predicted plus the ones that depend on it. 
    RFdf=RFdf.drop(drop_list, 1)
    #Find the NaN's
    nanlist=np.argwhere(np.isnan(RFdf.as_matrix()))
    namelist=list(RFdf)

    #Set all NaN's to the mean of the column
    for x in nanlist:
        RFdf.set_value(RFdf.iloc[[x[0]]].index[0],namelist[x[1]] , np.mean(RFdf[namelist[x[1]]]))
    #Now only columns with pure NaN's remain in the df. Remove those leaving only the good data.
    RFdf=RFdf.dropna(axis=1)
    featurelist=list(RFdf)
    #Define X and y
    X=RFdf.as_matrix()  
    #Use the one below if predicting amount of danes.
    if year=='All years':
        y=np.array(MLdf['pct_dansk'].tolist())
    else:
        y=np.array(MLdf['pct_dansk'][MLdf['aar']==year].tolist())
        
  
    #Use the one below if predicting high income
    #if year=='All years':
    #    y=np.array(MLdf['pct_hoj_indkomst'].tolist())
    #else:
    #    y=np.array(MLdf['pct_hoj_indkomst'][MLdf['aar']==year].tolist())
    
    #Use the one below if predicting age above 65
    #if year=='All years':
    #    y=np.array(MLdf['alder_pct_over_65'].tolist())
    #else:
    #    y=np.array(MLdf['alder_pct_over_65'][MLdf['aar']==year].tolist())
        
        
    #Remove nan's from y and X. This is only if there are nan's in y.
    naidx=np.isnan(y)
    y=y[~naidx]
    X=X[~naidx,]
    
    #Run Random forest
    kf=KFold(n=len(y), n_folds=nfold, shuffle=True, random_state=42)
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #Create random forest regressor
        tree=RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=5
                                   ,max_features='auto', max_leaf_nodes=None, bootstrap=True)
        #Fit random forest
        treefit.append(tree.fit(X_train,y_train))
        #Predict with random forest
        predtree=treefit[-1].predict(X_test)
        tree_performance_error_RMSE.append(mean_squared_error(y_test, predtree)**0.5)
    #Print the results for a given year.
    print "Year is: %s" %str(year) 
    print "Average RMSE is %.2f" %np.mean(tree_performance_error_RMSE)
    print "Std for RMSE is %.2f" %np.std(tree_performance_error_RMSE)
    year_error.append(np.mean(tree_performance_error_RMSE))
    
    #Find the most important features
    treezip=zip(treefit[0].feature_importances_,treefit[1].feature_importances_,treefit[2].feature_importances_,
       treefit[3].feature_importances_,treefit[4].feature_importances_,treefit[5].feature_importances_,
       treefit[6].feature_importances_,treefit[7].feature_importances_,treefit[8].feature_importances_,
       treefit[9].feature_importances_)
    #Get the mean of all the importances
    importances=[]
    for i in range(len(treezip)):
        importances.append(np.mean(treezip[i]))

    #Get the indices of the sorted important features. Get the highest first
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(10):
        print("%d. feature %s (%f)" % (f + 1, featurelist[indices[f]], importances[indices[f]]))
    print

Year is: 2008
Average RMSE is 5.77
Std for RMSE is 2.78
Feature ranking:
1. feature pct_ingen_udd (0.247082)
2. feature pct_tilkn_arb_markedet (0.223438)
3. feature alder_pct_6_17 (0.075423)
4. feature gns_boligareal_pr_pers (0.058795)
5. feature pct_lav_indkomst (0.045927)
6. feature bolig_pct_80_99_m2 (0.033573)
7. feature pct_enlige_m_born (0.028585)
8. feature alder_pct_0_5 (0.026988)
9. feature alder_pct_30_39 (0.024834)
10. feature pct_almen_bolig (0.019246)

Year is: 2009
Average RMSE is 5.60
Std for RMSE is 2.35
Feature ranking:
1. feature pct_ingen_udd (0.127237)
2. feature gns_boligareal_pr_pers (0.102590)
3. feature pct_mellemlang_viderg_udd (0.076732)
4. feature pct_enlige_u_born (0.046374)
5. feature pct_under_udd (0.045194)
6. feature pct_lang_viderg_udd (0.043031)
7. feature alder_pct_50_64 (0.041742)
8. feature pct_ovrige_husstande (0.041039)
9. feature alder_pct_18_29 (0.037913)
10. feature alder_pct_30_39 (0.036613)

Year is: 2010
Average RMSE is 5.91
Std for RMSE is 

# Elastic net

In [5]:
#Make function that standardize the data. This is needed to make sense of the magnitude of the coefficients.
stand=StandardScaler(copy=True, with_mean=True, with_std=True)

#Do nfold cross validation
nfold=10
year_error=[]

#Do the Elastic net (EN) for all the years
for year in yearlist:
    tree_performance_error_RMSE=[]
    treefit=[]
    #Determine what yeark is worked with.
    if year=='All years':
        RFdf=MLdf
    else:
        RFdf=MLdf[MLdf['aar']==year]
    #Use the one below if predicting amount of danes.
    drop_list=['aar','pct_dansk','pct_vestlig','pct_ikke_vestlig']
   
    #Use the one below if predicting high income
    #drop_list=['aar','pct_lav_indkomst','pct_middel_indkomst','pct_hoj_indkomst']
    
    #Use the one below if predicting age above 65
    #drop_list=['aar','alder_pct_0_5','alder_pct_6_17','alder_pct_18_29','alder_pct_30_39','alder_pct_40_49','alder_pct_50_64',
    #           'alder_pct_over_65']
    
    #Drop the feature that is being predicted plus the ones that depend on it. 
    RFdf=RFdf.drop(drop_list, 1)
    #Find the NaN's
    nanlist=np.argwhere(np.isnan(RFdf.as_matrix()))
    namelist=list(RFdf)

    #Set all NaN's to the mean of the column
    for x in nanlist:
        RFdf.set_value(RFdf.iloc[[x[0]]].index[0],namelist[x[1]] , np.mean(RFdf[namelist[x[1]]]))
    #Now only columns with pure NaN's remain in the df. Remove those leaving only the good data.
    RFdf=RFdf.dropna(axis=1)
    featurelist=list(RFdf)
    #Define X and y
    X=RFdf.as_matrix()
    
    #Use the one below if predicting amount of danes.
    if year=='All years':
        y=np.array(MLdf['pct_dansk'].tolist())
    else:
        y=np.array(MLdf['pct_dansk'][MLdf['aar']==year].tolist())
        
    #Use the one below if predicting high income
    #if year=='All years':
    #    y=np.array(MLdf['pct_hoj_indkomst'].tolist())
    #else:
    #    y=np.array(MLdf['pct_hoj_indkomst'][MLdf['aar']==year].tolist())
    
    #Use the one below if predicting age above 65
    #if year=='All years':
    #    y=np.array(MLdf['alder_pct_over_65'].tolist())
    #else:
    #    y=np.array(MLdf['alder_pct_over_65'][MLdf['aar']==year].tolist())
    
    #Remove nan's from y and X. This is only if there are nan's in y.
    naidx=np.isnan(y)
    y=y[~naidx]
    X=X[~naidx,]
    
    #Standardize data
    X=stand.fit_transform(X)
    
    #Do Elastic net
    EN_performance_error_RMSE=[]
    kf=KFold(n=len(y), n_folds=nfold, shuffle=True, random_state=42)
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #Create Elastic net model
        ENmodel=linear_model.ElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=True, normalize=False,
                                        precompute=False,
                                        max_iter=10000, tol=0.0001, warm_start=False, positive=False,
                                        random_state=None, selection='cyclic')
        #Fit Elastic net model
        ENmodelfit=ENmodel.fit(X_train,y_train)
        #Predict with Elastic net
        predEN=ENmodelfit.predict(X_test)
        EN_performance_error_RMSE.append(mean_squared_error(y_test, predEN)**0.5)
        
    #Print the results for a given year.
    print "Year is: %s" %str(year) 
    print "Average RMSE is %.2f" %np.mean(EN_performance_error_RMSE)
    print "Std for RMSE is %.2f" %np.std(EN_performance_error_RMSE)
    year_error.append(np.mean(EN_performance_error_RMSE))
    
    #Sort the features to get the largest ones.
    ENidx=np.argsort(ENmodelfit.coef_)[::-1]
    coeflist=ENmodelfit.coef_
    
    # Print the feature ranking (largest coefficients)
    print("Feature ranking:")
    for f in range(10):
        print("%d. feature %s (%f)" % (f + 1, featurelist[ENidx[f]], coeflist[ENidx[f]]))
    print

Year is: 2008
Average RMSE is 8.03
Std for RMSE is 5.24
Feature ranking:
1. feature pct_tilkn_arb_markedet (2.842675)
2. feature alder_pct_over_65 (2.090888)
3. feature pct_100_119_m2 (1.886862)
4. feature alder_pct_40_49 (1.829283)
5. feature pct_enlige_m_born (1.633036)
6. feature pct_hoj_indkomst (1.227872)
7. feature pct_off_ejet_bolig (1.076295)
8. feature pct_nettotilflytning (1.074760)
9. feature pct_erh_faglig_udd (1.029534)
10. feature pct_middel_indkomst (0.981932)

Year is: 2009
Average RMSE is 5.09
Std for RMSE is 1.24
Feature ranking:
1. feature pct_erh_faglig_udd (3.725106)
2. feature pct_under_udd (3.421109)
3. feature pct_enlige_u_born (3.188751)
4. feature pct_hoj_indkomst (2.921147)
5. feature pct_mellemlang_viderg_udd (2.770353)
6. feature pct_tilkn_arb_markedet (2.392964)
7. feature alder_pct_50_64 (2.309457)
8. feature alder_pct_30_39 (2.054215)
9. feature alder_pct_over_65 (1.933042)
10. feature pct__lonmodtagere (1.645161)

Year is: 2010
Average RMSE is 4.92
Std 